In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

np.random.seed(2018)

In [2]:
trn = pd.read_csv('data/train_ver2.csv')

/home1/irteam/.pyenv/versions/3.6.4/envs/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
tst = pd.read_csv('data/test_ver2.csv')

/home1/irteam/.pyenv/versions/3.6.4/envs/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
# remove no posessions

In [4]:
prods = trn.columns[24:].tolist()

In [5]:
# fillna for target values
trn[prods] = trn[prods].fillna(0.0).astype(np.int8)

In [6]:
no_product = trn[prods].sum(axis=1) == 0

In [7]:
trn = trn[~no_product]

In [8]:
for col in trn.columns[24:]:
    tst[col] = 0
    
# merge train and test
df = pd.concat([trn, tst], axis=0)

In [9]:
features = []

In [10]:
df.columns[:24].tolist()

['fecha_dato',
 'ncodpers',
 'ind_empleado',
 'pais_residencia',
 'sexo',
 'age',
 'fecha_alta',
 'ind_nuevo',
 'antiguedad',
 'indrel',
 'ult_fec_cli_1t',
 'indrel_1mes',
 'tiprel_1mes',
 'indresi',
 'indext',
 'conyuemp',
 'canal_entrada',
 'indfall',
 'tipodom',
 'cod_prov',
 'nomprov',
 'ind_actividad_cliente',
 'renta',
 'segmento']

In [11]:
# data cleaning
categorical_cols = ['ind_empleado', 'pais_residencia', 'sexo', 'tiprel_1mes', 'indresi', 'indext', 'conyuemp', 
                'canal_entrada', 'indfall', 'tipodom', 'nomprov', 'segmento']

for col in categorical_cols:
    df[col], _ = df[col].factorize(na_sentinel=-99)
    
features += categorical_cols

In [15]:
df['age'].replace(' NA', -99, inplace=True)
df['age'] = df['age'].astype(np.int8)

df['antiguedad'].replace('     NA', -99, inplace=True)
df['antiguedad'] = df['antiguedad'].astype(np.int8)

df['renta'].replace('         NA', -99, inplace=True)
df['renta'].fillna(-99, inplace=True)
df['renta'] = df['renta'].astype(float).astype(np.int8)

df['indrel_1mes'].replace('P', 5, inplace=True)
df['indrel_1mes'].fillna(-99, inplace=True)
df['indrel_1mes'] = df['indrel_1mes'].astype(float).astype(np.int8)

features += ['age','antiguedad','renta','ind_nuevo','indrel','indrel_1mes','ind_actividad_cliente']

In [16]:
## more feature engineering here

df['fecha_alta_month'] = df['fecha_alta'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8)
df['fecha_alta_year'] = df['fecha_alta'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[0])).astype(np.int16)
features += ['fecha_alta_month', 'fecha_alta_year']

df['ult_fec_cli_1t_month'] = df['ult_fec_cli_1t'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8)
df['ult_fec_cli_1t_year'] = df['ult_fec_cli_1t'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[0])).astype(np.int16)
features += ['ult_fec_cli_1t_month', 'ult_fec_cli_1t_year']

In [17]:
df.fillna(-99, inplace=True)

In [18]:
for col in df.columns:
    print(col)
    print(df[col].unique())

fecha_dato
['2015-01-28' '2015-02-28' '2015-03-28' '2015-04-28' '2015-05-28'
 '2015-06-28' '2015-07-28' '2015-08-28' '2015-09-28' '2015-10-28'
 '2015-11-28' '2015-12-28' '2016-01-28' '2016-02-28' '2016-03-28'
 '2016-04-28' '2016-05-28' '2016-06-28']
ncodpers
[1375586 1050611 1050612 ...,  660217  660231  660238]
ind_empleado
[  0 -99   1   2   3   4]
pais_residencia
[  0 -99   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16
  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34
  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52
  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70
  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88
  89  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106
 107 108 109 110 111 112 113 114 115 116 117]
sexo
[  0   1 -99]
age
[ 35  23  22  24  65  28  25  26  53  27  32  37  31  39  63  33  55  42
  58  38  50  30  45  44  36  29  60  57  67  47 -9

[1 0]
ind_cder_fin_ult1
[0 1]
ind_cno_fin_ult1
[0 1]
ind_ctju_fin_ult1
[0 1]
ind_ctma_fin_ult1
[0 1]
ind_ctop_fin_ult1
[0 1]
ind_ctpp_fin_ult1
[0 1]
ind_deco_fin_ult1
[0 1]
ind_deme_fin_ult1
[0 1]
ind_dela_fin_ult1
[0 1]
ind_ecue_fin_ult1
[0 1]
ind_fond_fin_ult1
[0 1]
ind_hip_fin_ult1
[0 1]
ind_plan_fin_ult1
[0 1]
ind_pres_fin_ult1
[0 1]
ind_reca_fin_ult1
[0 1]
ind_tjcr_fin_ult1
[0 1]
ind_valo_fin_ult1
[0 1]
ind_viv_fin_ult1
[0 1]
ind_nomina_ult1
[0 1]
ind_nom_pens_ult1
[0 1]
ind_recibo_ult1
[0 1]
fecha_alta_month
[ 1  8  0 10  7 12  9  3  4  2 11  5  6]
fecha_alta_year
[2015 2012    0 2014 2013 2005 2002 2004 2003 2006 2007 2011 2009 2010 2001
 1999 2008 1997 1998 2000 1996 1995 2016]
ult_fec_cli_1t_month
[ 0  7  8  9 10 11 12  1  2  3  4  5  6]
ult_fec_cli_1t_year
[   0 2015 2016]


In [19]:
def date_to_int(str_date):
    Y, M, D = [int(a) for a in str_date.strip().split("-")] # "2016-05-28"
    int_date = (int(Y) - 2015) * 12 + int(M)
    return int_date

df['int_date'] = df['fecha_dato'].map(date_to_int).astype(np.int8)

In [20]:
df_lag = df.copy()

In [21]:
df_lag.columns = [col + '_prev' if col not in ['ncodpers', 'int_date'] else col for col in df.columns ]

In [22]:
df_lag['int_date'] += 1

In [23]:
df_trn = df.merge(df_lag, on=['ncodpers','int_date'], how='left')

In [24]:
df.shape, df_lag.shape, df_trn.shape

((12020685, 53), (12020685, 53), (12020685, 104))

In [25]:
df_trn.fillna(-99, inplace=True)

In [26]:
features += [feature + '_prev' for feature in features]

In [27]:
features += [prod + '_prev' for prod in prods]

In [28]:
trn = df_trn[~df_trn['fecha_dato'].isin(['2016-05-28', '2016-06-28'])]
tst = df_trn[df_trn['fecha_dato'] == '2016-05-28']
submit = df_trn[df_trn['fecha_dato'] == '2016-06-28']

In [29]:
trn.shape, tst.shape, submit.shape

((10394531, 104), (696539, 104), (929615, 104))

In [30]:
X = []
Y = []
for i, prod in enumerate(prods):
    prev = prod + '_prev'
    prX = trn[(trn[prod] == 1) & (trn[prev] == 0)]
    prY = np.zeros(prX.shape[0], dtype=np.int8) + i
    X.append(prX)
    Y.append(prY)

In [31]:
XY = pd.concat(X)

In [32]:
Y = np.hstack(Y)

In [33]:
XY['y'] = Y

In [34]:
mask = np.random.rand(len(XY)) < 0.8
XY_trn = XY[mask]
XY_vld = XY[~mask]

In [67]:
ncodpers_tst = tst.as_matrix(columns=['ncodpers'])
ncodpers_submit = submit.as_matrix(columns=['ncodpers'])

In [36]:
for prod in prods:
    prev = prod + '_prev'
    padd = prod + '_add'
    tst[padd] = tst[prod] - tst[prev]

/home1/irteam/.pyenv/versions/3.6.4/envs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [37]:
add_tst = tst.as_matrix(columns=[prod + '_add' for prod in prods])
add_tst_list = [list() for i in range(len(ncodpers_tst))]

In [38]:
count_tst = 0
for ncodper in range(len(ncodpers_tst)):
    for prod in range(len(prods)):
        if add_tst[ncodper, prod] > 0:
            add_tst_list[ncodper].append(prod)
            count_tst += 1

In [39]:
import numpy as np

def apk(actual, predicted, k=7, default=0.0):
    # MAP@7 이므로, 최대 7개만 사용한다
    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        # 점수를 부여하는 조건은 다음과 같다 :
        # 예측값이 정답에 있고 (‘p in actual’)
        # 예측값이 중복이 아니면 (‘p not in predicted[:i]’) 
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    # 정답값이 공백일 경우, 무조건 1.0점을 반환한다
    if not actual:
        return default

    # 정답의 개수(len(actual))로 average precision을 구한다
    return score / min(len(actual), k)

def mapk(actual, predicted, k=7, default=0.0):
    # list of list인 정답값(actual)과 예측값(predicted)에서 고객별 Average Precision을 구하고, np.mean()을 통해 평균을 계산한다
    return np.mean([apk(a, p, k, default) for a, p in zip(actual, predicted)])

In [40]:
mapk(add_tst_list, add_tst_list, 7, 0.0)

0.042663799155539028

In [48]:
import xgboost as xgb

param = {
    'booster': 'gbtree',
    'max_depth': 8,
    'nthread': 24,
    'num_class': len(prods),
    'objective': 'multi:softprob',
    'silent': 1,
    'eval_metric': 'mlogloss',
    'eta': 0.1,
    'min_child_weight': 10,
    'colsample_bytree': 0.8,
    'colsample_bylevel': 0.9,
    'seed': 2018,
    }

In [42]:
X_trn = XY_trn.as_matrix(columns=features)
Y_trn = XY_trn.as_matrix(columns=['y'])
dtrn = xgb.DMatrix(X_trn, label=Y_trn, feature_names=features)

In [43]:
X_vld = XY_vld.as_matrix(columns=features)
Y_vld = XY_vld.as_matrix(columns=['y'])
dvld = xgb.DMatrix(X_vld, label=Y_vld, feature_names=features)

In [49]:
watch_list = [(dtrn, 'train'), (dvld, 'eval')]
model = xgb.train(param, dtrn, num_boost_round=1000, evals=watch_list, early_stopping_rounds=20)

[0]	train-mlogloss:2.79508	eval-mlogloss:2.79615
Multiple eval metrics have been passed: 'eval-mlogloss' will be used for early stopping.

Will train until eval-mlogloss hasn't improved in 20 rounds.
[1]	train-mlogloss:2.56889	eval-mlogloss:2.57081
[2]	train-mlogloss:2.39626	eval-mlogloss:2.39868
[3]	train-mlogloss:2.2642	eval-mlogloss:2.26725
[4]	train-mlogloss:2.16456	eval-mlogloss:2.16852
[5]	train-mlogloss:2.07093	eval-mlogloss:2.07553
[6]	train-mlogloss:1.99467	eval-mlogloss:1.99991
[7]	train-mlogloss:1.92661	eval-mlogloss:1.93244
[8]	train-mlogloss:1.86707	eval-mlogloss:1.87344
[9]	train-mlogloss:1.81729	eval-mlogloss:1.82424
[10]	train-mlogloss:1.77208	eval-mlogloss:1.77969
[11]	train-mlogloss:1.72986	eval-mlogloss:1.73812
[12]	train-mlogloss:1.69358	eval-mlogloss:1.70251
[13]	train-mlogloss:1.65966	eval-mlogloss:1.66912
[14]	train-mlogloss:1.63031	eval-mlogloss:1.64045
[15]	train-mlogloss:1.60192	eval-mlogloss:1.61256
[16]	train-mlogloss:1.57589	eval-mlogloss:1.58708
[17]	train

[161]	train-mlogloss:1.17721	eval-mlogloss:1.24802
[162]	train-mlogloss:1.17663	eval-mlogloss:1.24793
[163]	train-mlogloss:1.17623	eval-mlogloss:1.24792
[164]	train-mlogloss:1.17574	eval-mlogloss:1.24784
[165]	train-mlogloss:1.17527	eval-mlogloss:1.24776
[166]	train-mlogloss:1.17477	eval-mlogloss:1.24769
[167]	train-mlogloss:1.17419	eval-mlogloss:1.24756
[168]	train-mlogloss:1.17371	eval-mlogloss:1.24752
[169]	train-mlogloss:1.17321	eval-mlogloss:1.24742
[170]	train-mlogloss:1.1728	eval-mlogloss:1.2474
[171]	train-mlogloss:1.17215	eval-mlogloss:1.24726
[172]	train-mlogloss:1.17168	eval-mlogloss:1.24723
[173]	train-mlogloss:1.17114	eval-mlogloss:1.24715
[174]	train-mlogloss:1.17066	eval-mlogloss:1.2471
[175]	train-mlogloss:1.17005	eval-mlogloss:1.24697
[176]	train-mlogloss:1.16964	eval-mlogloss:1.24694
[177]	train-mlogloss:1.16918	eval-mlogloss:1.24692
[178]	train-mlogloss:1.16869	eval-mlogloss:1.24687
[179]	train-mlogloss:1.16828	eval-mlogloss:1.24681
[180]	train-mlogloss:1.16784	eval-

NameError: name 'pickle' is not defined

In [50]:
import pickle
pickle.dump(model, open("next_multi.pickle", "wb"))

In [51]:
best_ntree_limit = model.best_ntree_limit

In [52]:
best_ntree_limit

300

In [53]:
X = XY.as_matrix(columns=features)
Y = XY.as_matrix(columns=['y'])
dX = xgb.DMatrix(X, label=Y, feature_names=features)

In [54]:
best_ntree_limit = int(best_ntree_limit * (len(XY_trn) + len(XY_vld)) / len(XY_trn))

In [ ]:
evallist = [(dX, 'trnvld')]
model = xgb.train(param, dX, best_ntree_limit, evals=evallist)

In [55]:
print("Feature importance:")
for kv in sorted([(k,v) for k,v in model.get_fscore().items()], key=lambda kv: kv[1], reverse=True):
    print(kv)

Feature importance:
('renta', 42982)
('antiguedad', 27830)
('antiguedad_prev', 26261)
('age', 26238)
('age_prev', 23474)
('fecha_alta_month', 21929)
('nomprov', 16409)
('fecha_alta_year', 16186)
('canal_entrada', 13349)
('renta_prev', 12254)
('nomprov_prev', 11154)
('canal_entrada_prev', 8944)
('fecha_alta_month_prev', 6534)
('ind_recibo_ult1_prev', 5745)
('fecha_alta_year_prev', 5337)
('ind_cco_fin_ult1_prev', 5301)
('ind_ecue_fin_ult1_prev', 5237)
('sexo', 5098)
('ind_cno_fin_ult1_prev', 4526)
('segmento', 3888)
('ind_tjcr_fin_ult1_prev', 3806)
('segmento_prev', 3577)
('ind_reca_fin_ult1_prev', 3521)
('ind_dela_fin_ult1_prev', 2821)
('ind_ctop_fin_ult1_prev', 2762)
('ind_nom_pens_ult1_prev', 2740)
('ind_valo_fin_ult1_prev', 2291)
('ind_nomina_ult1_prev', 2290)
('ind_ctpp_fin_ult1_prev', 2171)
('sexo_prev', 1874)
('ind_fond_fin_ult1_prev', 1773)
('tiprel_1mes_prev', 1665)
('ind_ctma_fin_ult1_prev', 1560)
('ind_actividad_cliente', 1537)
('ind_actividad_cliente_prev', 1426)
('tiprel_1me

In [56]:
X_test = tst.as_matrix(columns=features)
dtest = xgb.DMatrix(X_test, feature_names=features)

preds = model.predict(dtest, ntree_limit=best_ntree_limit)

In [57]:
preds

array([[  1.27775329e-05,   1.41081036e-05,   2.14116899e-05, ...,
          2.59958953e-02,   2.59767491e-02,   6.32383347e-01],
       [  1.25464521e-05,   9.96616836e-06,   1.60573982e-05, ...,
          2.34538540e-02,   2.25624815e-02,   6.63069665e-01],
       [  4.12882036e-05,   5.50963960e-05,   4.90546256e-01, ...,
          2.81150278e-04,   1.37237948e-04,   1.29365391e-04],
       ..., 
       [  2.20387628e-05,   2.15961154e-05,   2.10560047e-05, ...,
          8.32424685e-02,   8.35903063e-02,   6.25931501e-01],
       [  8.43750422e-06,   9.31614068e-06,   1.13008400e-05, ...,
          3.06048952e-02,   3.04748137e-02,   8.56111705e-01],
       [  1.42435383e-05,   1.57267841e-05,   5.44787884e-01, ...,
          5.21923564e-02,   1.38039082e-01,   6.74095601e-02]], dtype=float32)

In [58]:
# subtract prev1 because possessed item cannot be bought
preds = preds - tst.as_matrix(columns=[prod + '_prev' for prod in prods])

In [59]:
ncodpers_tst.shape

(696539, 1)

In [60]:
result = []

for ncodper, pred in zip(ncodpers_tst, preds):
    y_prods = [(y,p,ip) for y,p,ip in zip(pred, prods, range(len(prods)))]
    y_prods = sorted(y_prods, key=lambda a: a[0], reverse=True)[:7]
    result.append([ip for y,p,ip in y_prods])

Exception ignored in: <bound method Booster.__del__ of <xgboost.core.Booster object at 0x7faa0e88a550>>
Traceback (most recent call last):
  File "/home1/irteam/.pyenv/versions/3.6.4/envs/venv/lib/python3.6/site-packages/xgboost/core.py", line 745, in __del__
    if self.handle is not None:
AttributeError: 'Booster' object has no attribute 'handle'


In [61]:
mapk(add_tst_list, result, 7, 0.0)

0.036981821651664248

In [62]:
X_submit = submit.as_matrix(columns=features)
dsubmit = xgb.DMatrix(X_submit, feature_names=features)

preds_submit = model.predict(dsubmit, ntree_limit=best_ntree_limit)

In [63]:
# subtract prev1 because possessed item cannot be bought
preds_submit = preds_submit - submit.as_matrix(columns=[prod + '_prev' for prod in prods])

In [78]:
ncodpers_submit[929561]

array([660261])

In [79]:
# write submission file
submit_file = open('output/baseline.xgb', 'w')
submit_file.write('ncodpers,added_products\n')
for ncodper, pred in zip(ncodpers_submit, preds_submit):
    y_prods = [(y,p,ip) for y,p,ip in zip(pred, prods, range(len(prods)))]
    y_prods = sorted(y_prods, key=lambda a: a[0], reverse=True)[:7]
    y_prods = [p for y,p,ip in y_prods]
    submit_file.write('{},{}\n'.format(int(ncodper), ' '.join(y_prods)))

- private leaderboard 0.025018
- rank : 1001